In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
df_road_network = pd.read_csv('./data/road_network_sub-dataset.csv',sep='\t')
df_road_network

,link_id,width,direction,snodeid,enodeid,length,speedclass,lanenum
0,1562548955,30,3,1520445066,1549742690,0.038,6,1
1,1570012174,55,2,1520482305,1554157827,0.013,5,2
2,1542179399,30,2,1520491527,1531173821,0.146,6,1
3,1562548950,55,1,1520433538,1549742156,0.029,7,1
4,1526068509,30,2,1520486391,1520499209,0.156,6,1
...,...,...,...,...,...,...,...,...
45143,1542174850,55,1,1531470215,1531432689,0.028,7,1
45144,1526066032,30,2,1520457318,1520455758,0.112,6,1
45145,1542091409,55,1,1520434054,1530823987,0.083,7,1
45146,1525859466,55,2,1520406288,1520411419,0.020,5,2


In [5]:
adj_matrix = np.zeros((45148, 45148))
# 遍历列 A 的数据
for index, row in df_road_network.iterrows():
    current_value1 = row['snodeid']
    current_value2 = row['enodeid']         
    duplicate_indices_pre = df_road_network[df_road_network['enodeid'] == current_value1].index.tolist()
    for adj_ind in set(duplicate_indices_pre):
        adj_ind_dir = df_road_network.loc[adj_ind,'direction']
        if row['direction'] == 0 or row['direction'] == 1:
            if adj_ind_dir==0 or adj_ind_dir==1:
                adj_matrix[index][adj_ind] = 1
                adj_matrix[adj_ind][index] = 1
            elif adj_ind_dir==2:
                adj_matrix[adj_ind][index] = 1
            else:
                adj_matrix[index][adj_ind] = 1   
        elif row['direction'] == 2:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==2:
                adj_matrix[adj_ind][index] = 1
        else:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==3:
                adj_matrix[index][adj_ind] = 1
                
    duplicate_indices_post = df_road_network[df_road_network['snodeid'] == current_value2].index.tolist()
    for adj_ind in set(duplicate_indices_post):
        adj_ind_dir = df_road_network.loc[adj_ind,'direction']
        if row['direction'] == 0 or row['direction'] == 1:
            if adj_ind_dir==0 or adj_ind_dir==1:
                adj_matrix[index][adj_ind] = 1
                adj_matrix[adj_ind][index] = 1
            elif adj_ind_dir==2:
                adj_matrix[index][adj_ind] = 1
            else:
                adj_matrix[adj_ind][index] = 1   
        elif row['direction'] == 2:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==2:
                adj_matrix[index][adj_ind] = 1
        else:
            if adj_ind_dir==0 or adj_ind_dir==1 or adj_ind_dir==3:
                adj_matrix[adj_ind][index] = 1
adj_matrix.shape     

(45148, 45148)

In [6]:
# 依据conn邻接矩阵生成eigenmaps,用于生成空间嵌入
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.linalg import eigh
def compute_eigenmaps(adj_mx, k):
    A = adj_mx.copy()
    # row, col = A.nonzero()
    # A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

    n_components,labels = connected_components(csr_matrix(A), directed=True, return_labels=True)
    assert n_components == 1  # the graph should be connected

    n = A.shape[0]
    A = zero_diagonals(A)
    # degrees = np.sum(A, axis=1)
    # # 避免除以零，将度为零的节点的度设置为一个小正数（例如1e-6）
    # degrees[degrees == 0] = 1e-6
    # D = degrees**(-1/2)
    # L = np.eye(n) - (A * D).T * D  # normalized Laplacian
    D = np.sum(A, axis=1)**(-1/2)
    L = np.eye(n) - (A * D).T * D  # normalized Laplacian

    _, v = eigh(L)
    print('pre_eigenmaps.shape:',v.shape)
    eigenmaps = v[:, 1:(k + 1)]  # eigenvectors corresponding to the k smallest non-trivial eigenvalues

    return eigenmaps

def zero_diagonals(x):
    y = x.copy()
    y[np.diag_indices_from(y)] = 0

    return y

In [7]:
A = adj_matrix.copy()
# row, col = A.nonzero()
# A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric
n_components,labels = connected_components(csr_matrix(A), directed=True, connection='strong', return_labels=True)
print(n_components)
print(len(labels))

20849
45148


In [8]:
# 使用unique函数获取唯一的值以及它们的计数
unique_values, counts = np.unique(labels, return_counts=True)
unique_values

array([    0,     1,     2, ..., 20846, 20847, 20848], dtype=int32)

In [9]:
sl = sorted(list(counts),reverse=True)
sl[:10]

[6392, 3848, 2307, 600, 537, 432, 278, 232, 231, 218]

In [10]:
np.where(counts==6392)

(array([1298]),)

In [11]:
indices = np.where(labels == 1298)
indices_ = indices[0]
indices_

array([    2,     5,     8, ..., 45133, 45134, 45137])

In [12]:
len(indices_)

6392

In [12]:
road_network_linkid_filtered = df_road_network.loc[indices_, 'link_id']
len(road_network_linkid_filtered)

6392

In [3]:
with open('./data/road_network_linkid_filtered_6392.pkl', 'wb') as file:
    pickle.dump(list(road_network_linkid_filtered), file)
file.close()

NameError: name 'road_network_linkid_filtered' is not defined

In [4]:
road_network_linkid_filtered = pd.read_pickle('./data/road_network_linkid_filtered_6392.pkl')
len(road_network_linkid_filtered)

6392

In [13]:
# 使用索引数组从大矩阵中取出行和列
adj_matrix_filtered = adj_matrix[indices_][:, indices_]
adj_matrix_filtered.shape

(6392, 6392)

In [14]:
with open('./data/adj_matrix_filtered_6392.pkl', 'wb') as file:
    pickle.dump(adj_matrix_filtered, file)
file.close()

In [17]:
adj_matrix_filtered = pd.read_pickle('./data/adj_matrix_filtered_6392.pkl')
adj_matrix_filtered.shape

(6392, 6392)

In [18]:
len(adj_matrix_filtered[adj_matrix_filtered==1])

8928

In [19]:
with open('./data/indices_6392.pkl', 'wb') as file:
    pickle.dump(indices_, file)
file.close()

In [20]:
indices_6392 = pd.read_pickle('./data/indices_6392.pkl')
indices_6392

array([    2,     5,     8, ..., 45133, 45134, 45137])

In [21]:
len(indices_6392)

6392